# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
cityData_df = pd.read_csv("cityData.csv")
cityData_df.head()


,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,sirnak,37.52,42.46,48.09,50,1013,6.02,100
1,bundaberg,-24.85,152.35,66.31,48,1021,9.17,90
2,atuona,-9.80,-139.03,82.04,73,1010,17.40,13
3,port elizabeth,-33.92,25.57,59.00,100,1025,4.70,90
4,qaanaaq,77.48,-69.36,20.88,88,997,2.82,97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Store latitude and longitude in locations
locations = cityData_df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humid = cityData_df["humidity"]


In [152]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID", layout = figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True)


heat_layer.max_intensity = 100
heat_layer.point_radius = 20

heat_layer.gradient = [
    (200, 200, 200, 0.6),
    (100, 100,100, 0.3),
    (50, 50, 50, 0.3)
]

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [87]:
# Filter cities to fit weather conditions
temp_mask = (df.temperature >= 70) & (df.temperature < 80)
wind_mask = df.wind_speed < 10
cloud_mask = df.cloudiness == 0
all_mask = temp_mask & wind_mask & cloud_mask

# Create data frame with ideal weather conditions
ideal_cities = df.loc[all_mask].reset_index(drop=True)
ideal_cities

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,pasan,22.85,82.20,76.95,36,1007,3.38,0
1,mecca,21.43,39.83,77.04,23,1008,2.42,0
2,sao filipe,14.90,-24.50,77.74,55,1014,9.08,0
3,nioro,13.35,-15.75,75.07,47,1010,6.49,0
4,nioki,-2.72,17.68,74.23,73,1010,1.68,0
5,chuzhou,32.32,118.30,79.83,65,1008,2.44,0
6,namibe,-15.20,12.15,77.43,58,1011,2.66,0
7,riyadh,24.69,46.72,75.20,27,1015,3.36,0


In [88]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = ideal_cities[["latitude", "longitude"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [90]:
# Set up info box for the city markers
city_info = []

#for loop to get hotel data for info box
for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Name</dt><dd>{row.cities}</dd>
                <dt>Temp</dt><dd>{row.temperature}</dd>
                <dt>Cloudiness</dt><dd>{row.cloudiness}%</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [91]:
# Customize the size of the map figure with markers and info box
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = ideal_cities[["latitude", "longitude"]]

# Setup marker and info box
marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [110]:
# Get first item 
ideal_cities.head(1)

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,pasan,22.85,82.2,76.95,36,1007,3.38,0


In [132]:
# Set up a parameters dictionary
params = {"key": g_key}

#base url
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Extract latitude and longitude
lat = ideal_cities["latitude"][1]
lng = ideal_cities["longitude"][1]

# Update address key value
params['latlng'] = f"{lat},{lng}"

# make request using params dictionary
hotel = requests.get(base_url, params=params)

# Get first item
hotel = hotel.json()
hotel["results"][0]["address_components"]

#Loop to get country name
for addressComp in hotel["results"][0]["address_components"]:
    if addressComp["types"][0] == "country":
        print(addressComp["long_name"])

Saudi Arabia


In [136]:
# Create function to get country for each city
def getCountryForCity(lat, long):    
    rtnCountry = ""
    
    # Set up a parameters dictionary    
    params = {"key": g_key}

    # base url
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    # Extract latitude and longitude
    lat = ideal_cities["latitude"][indx]
    lng = ideal_cities["longitude"][indx]

    # update address key value
    params['latlng'] = f"{lat},{lng}"

    # make request
    hotel = requests.get(base_url, params=params)

    hotel = hotel.json()
    hotel["results"][0]["address_components"]


    for addressComp in hotel["results"][0]["address_components"]:
        if addressComp["types"][0] == "country":
            rtnCountry = addressComp["long_name"]
            
    return rtnCountry

In [137]:
# Get countries
countries = []
for indx, row in ideal_cities.iterrows():
    countries.append(getCountryForCity(row.latitude, row.longitude))

In [119]:
countries

['India',
 'Saudi Arabia',
 'Cape Verde',
 'The Gambia',
 'Democratic Republic of the Congo',
 'China',
 'Angola',
 'Saudi Arabia']

In [120]:
# Set up a parameters dictionary 
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Extract latitude and longitude
lat = ideal_cities["latitude"][0]
lng = ideal_cities["longitude"][0]

#set up location parameter
params["location"] = f"{lat},{lng}"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Get hotel name
name_address = requests.get(base_url, params=params)
name_address = name_address.json()

name_address["results"][0]["name"]

'Yadav Hotel'

In [121]:
#Extract info data for first hotel
name_address["results"][0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 22.844035, 'lng': 82.1991015},
  'viewport': {'northeast': {'lat': 22.84538777989272,
    'lng': 82.20044252989271},
   'southwest': {'lat': 22.84268812010728, 'lng': 82.19774287010728}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
 'id': '474beb01ced60989dac3e3683fd50be556eaf58b',
 'name': 'Yadav Hotel',
 'place_id': 'ChIJ22vZHvbfhzkRcoBnQ_dKVbA',
 'plus_code': {'compound_code': 'R5VX+JJ Pasan, Chhattisgarh, India',
  'global_code': '7MJ4R5VX+JJ'},
 'rating': 0,
 'reference': 'ChIJ22vZHvbfhzkRcoBnQ_dKVbA',
 'scope': 'GOOGLE',
 'types': ['lodging', 'point_of_interest', 'establishment'],
 'user_ratings_total': 0,
 'vicinity': 'Pasan'}

In [122]:
# Use the lat/lng we recovered to identify airports
names = []
addresses = []
ratings = []

# Set up a parameters dictionary 
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# For loop to get information for all hotels
for index, row in ideal_cities.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and print url
    name_address = requests.get(base_url, params=params)

    # Convert to json
    name_address = name_address.json()
    
    # Skip missing fields and continue loop
    try:
        names.append(name_address["results"][0]["name"])
        addresses.append(name_address["results"][0]["vicinity"])
        ratings.append(name_address["results"][0]["rating"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append("")
        addresses.append("")
        ratings.append("")

Missing field/result... skipping.
Missing field/result... skipping.


In [123]:
# Call hotel names
names

['Yadav Hotel',
 'Hilton Makkah Convention Hotel',
 'Hotel Ocean View & Restaurante Seafood',
 '',
 '',
 'Wyndham Grand Plaza Royale Kaidi',
 'Hotel ibis Styles Iu Namibe',
 'Four Seasons Hotel Riyadh At Kingdom Center']

In [124]:
# Hotel info data
ideal_cities["country"] = countries
ideal_cities["hotels"] = names
ideal_cities["addresses"] = addresses
ideal_cities["ratings"] = ratings

ideal_cities

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness,country,hotels,addresses,ratings
0,pasan,22.85,82.20,76.95,36,1007,3.38,0,India,Yadav Hotel,Pasan,0
1,mecca,21.43,39.83,77.04,23,1008,2.42,0,Saudi Arabia,Hilton Makkah Convention Hotel,"Ibrahim Al Khalil, Jabal Omar، Mecca",4.4
2,sao filipe,14.90,-24.50,77.74,55,1014,9.08,0,Cape Verde,Hotel Ocean View & Restaurante Seafood,Bila Baixo Ilha do Fogo، São Filipe,4.5
3,nioro,13.35,-15.75,75.07,47,1010,6.49,0,The Gambia,,,
4,nioki,-2.72,17.68,74.23,73,1010,1.68,0,Democratic Republic of the Congo,,,
5,chuzhou,32.32,118.30,79.83,65,1008,2.44,0,China,Wyndham Grand Plaza Royale Kaidi,"2888 South Nanqiao Road, Chuzhou",4.5
6,namibe,-15.20,12.15,77.43,58,1011,2.66,0,Angola,Hotel ibis Styles Iu Namibe,"Avenida Eduardo Mondlane, Namibe",3.6
7,riyadh,24.69,46.72,75.20,27,1015,3.36,0,Saudi Arabia,Four Seasons Hotel Riyadh At Kingdom Center,"Dist.،، Kingdom Centre, Olaya Street، Riyadh",4.5


In [138]:
# Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humid = df["humidity"]

In [139]:
# For loop for info box
city_info = []

for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Hotel Name</dt><dd>{row.hotels}</dd>
                <dt>City</dt><dd>{row.cities}</dd>
                <dt>Country</dt><dd>{row.country}</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [153]:
# Heat map layout
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(map_type="ROADMAP", layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True)


# Add layer
fig.add_layer(heat_layer)

#add markers and info boxes
coordinates = ideal_cities[["latitude", "longitude"]]
marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…